# Assignment 01

## 1. Background and Motivation
Pretraining foundation models requires large-scale, diverse, and high-quality datasets. Raw text data often contains duplicates, noise, or formatting issues that can negatively impact model learning. Effective preprocessing—including cleaning, normalization, tokenization, and batching—is crucial for training eﬀicient, reliable models.


## 2. Learning Objectives

By completing this assignment, students will be able to:

1. Identify and access **publicly available large-scale text datasets** suitable for pretraining.
2. Implement **data cleaning and normalization pipelines**, including deduplication and low-quality content removal.
3. Apply **tokenization strategies** suitable for transformer-based foundation models.
4. Develop **custom data loaders** in PyTorch or TensorFlow for eﬀicient batch training.

In [2]:
# libs
import torch
from datasets import load_dataset
from transformers import AutoTokenizer
from tqdm import tqdm
import json
import os
import hashlib
import re
import random
from torch.utils.data import IterableDataset, DataLoader

/Users/zhenting/miniconda3/envs/7374_LLM/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 3. Dataset Selection for Foundation Model Pre-Training

I selected three diverse datasets to meet the multi-domain requirement (encyclopedic, news, general web text) for pre-training data collection.

---

### 🔍 Overview Table

| Feature | CC-News (Original) | Wikipedia (Wikimedia) | OpenWebText |
| :--- | :--- | :--- | :--- |
| **Domain** | News | Encyclopedic | General Web |
| **HuggingFace Path** | [`cc_news`](https://huggingface.co/datasets/cc_news) | [`wikimedia/wikipedia`](https://huggingface.co/datasets/wikimedia/wikipedia) | [`Skylion007/openwebtext`](https://huggingface.co/datasets/Skylion007/openwebtext) |
| **Temporal Range** | Jan 2017 – Dec 2019 | November 2023 dump | 2019 (Reddit-sourced) |
| **Volume** | ~708k English articles | 6.4M English articles | 8M+ documents (24.2 GB) |
| **Pre-processing** | **Raw (Requires Deduplication)** | Markdown/references stripped | Deduplicated, English-filtered |
| **Status** | Ready for cleaning | Ready for normalization | Ready for normalization |

---

### 🛠️ Processing Requirements

#### 1. CC-News (Original) — News Domain
* **Description:** A dataset containing news articles from news sites all over the world, spanning Jan 2017 to Dec 2019. 
* **Implementation Steps:**
    * [ ] **Deduplication (Required)**
    * [x] Text normalization (lowercase, whitespace)
    * [x] Short document filtering (< 50 words)
    * [x] Symbol/noise removal

#### 2. Wikipedia (Wikimedia) — Encyclopedic Domain
* **Description:** Cleaned English Wikipedia articles with markdown and reference sections already removed.
* **Implementation Steps:**
    * [x] Text normalization (lowercase, whitespace)
    * [x] Short document filtering (< 50 words)
    * [x] Symbol/noise removal

#### 3. OpenWebText — General Web Domain
* **Description:** Open-source replication of OpenAI's WebText dataset, sourced from Reddit-upvoted web pages.
* **Implementation Steps:**
    * [x] Text normalization (lowercase, whitespace)
    * [x] Short document filtering (< 50 words)
    * [x] Symbol/noise removal

In [2]:
output_file = "/Users/zhenting/7374_LLM/Assignment_01/raw_dataset.jsonl"

if os.path.exists(output_file):
    os.remove(output_file)

# Sample sizes
news_samples = 100000
wiki_samples = 100000
web_samples = 100000

# Helper function to append to file immediately to save RAM
def stream_to_file(dataset, num_samples, domain_name, pbar_desc):
    with open(output_file, "a", encoding="utf-8") as f:
        iterator = dataset["train"].take(num_samples)
        for ex in tqdm(iterator, total=num_samples, desc=pbar_desc):
            record = {
                "text": ex["text"],
                "domain": domain_name
            }
            f.write(json.dumps(record, ensure_ascii=False) + "\n")

# 1. Collect News
print("Loading CC-News...")
cc_news = load_dataset("cc_news", streaming=True)
stream_to_file(cc_news, news_samples, "news", "News")

# 2. Collect Wikipedia
print("Loading Wikipedia...")
wiki = load_dataset("wikimedia/wikipedia", "20231101.en", streaming=True)
stream_to_file(wiki, wiki_samples, "encyclopedic", "Wikipedia")

# 3. Collect OpenWebText
print("Loading OpenWebText...")
openwebtext = load_dataset("Skylion007/openwebtext", streaming=True)
stream_to_file(openwebtext, web_samples, "web", "WebText")

total_bytes = os.path.getsize(output_file)
total_gb = total_bytes / (1024 ** 3)

Loading CC-News...


News: 100%|█████████████████████████████████████| 100000/100000 [00:12<00:00, 7716.14it/s]


Loading Wikipedia...


Wikipedia: 100%|████████████████████████████████| 100000/100000 [00:18<00:00, 5525.45it/s]


Loading OpenWebText...


WebText: 100%|██████████████████████████████████| 100000/100000 [00:15<00:00, 6548.94it/s]

Total dataset size: 1.13 GB
Data saved to /Users/zhenting/7374_LLM/raw_dataset.jsonl


## 4. Preprocessing Requirements

### 4.1 Cleaning
* Remove duplicate documents.
* Normalize text: lowercase, remove extra whitespace, strip irrelevant symbols.
* Remove low-quality or very short documents (e.g., fewer than 50 words).
* Optionally remove HTML tags, markdown, or reference markers.

In [5]:
input_file = "/Users/zhenting/7374_LLM/Assignment_01/raw_dataset.jsonl"
output_file = "/Users/zhenting/7374_LLM/Assignment_01/clean_dataset.jsonl"

if not os.path.exists(input_file):
    print(f"Error: {input_file} not found!")
else:
    stats = {
        "total_processed": 0,
        "duplicates_removed": 0,
        "short_docs_removed": 0,
        "kept": 0
    }

    seen_hashes = set()
    print(f"Starting preprocessing pipeline on {input_file}...")
    total_lines = sum(1 for _ in open(input_file, encoding="utf-8"))

    with open(input_file, "r", encoding="utf-8") as fin, \
         open(output_file, "w", encoding="utf-8") as fout:
        
        for line in tqdm(fin, total=total_lines, desc="Cleaning"):
            stats["total_processed"] += 1
            
            try:
                doc = json.loads(line)
                text = doc.get("text", "")
                
                # Normalize text: lowercase, remove extra whitespace
                text_clean = re.sub(r'\s+', ' ', text.lower().strip())
                
                # Remove duplicate documents
                # Ex: 'Hello' -> 8b1a9953c4611296a827abf8c47804d7, easier to compare
                text_hash = hashlib.md5(text_clean.encode("utf-8")).hexdigest()
                if text_hash in seen_hashes:
                    stats["duplicates_removed"] += 1
                    continue
                seen_hashes.add(text_hash)
                
                # Remove low-quality or very short documents (< 50 words)
                word_count = len(text_clean.split())
                if word_count < 50:
                    stats["short_docs_removed"] += 1
                    continue
                
                doc["text"] = text_clean
                fout.write(json.dumps(doc, ensure_ascii=False) + "\n")
                stats["kept"] += 1
                
            except json.JSONDecodeError:
                continue

    clean_size_gb = os.path.getsize(output_file) / (1024 ** 3)
    print(f"Original Docs: {stats['total_processed']}")
    print(f"Duplicates Removed: {stats['duplicates_removed']}")
    print(f"Short Docs Removed (<50 words): {stats['short_docs_removed']}")
    print(f"Final Docs Kept: {stats['kept']}")
    print(f"Final Dataset Size: {clean_size_gb:.2f} GB")
    print(f"Cleaned data saved to: {output_file}")

Starting preprocessing pipeline on /Users/zhenting/7374_LLM/raw_dataset.jsonl...


Cleaning: 100%|█████████████████████████████████| 300000/300000 [00:36<00:00, 8127.49it/s]


=== Preprocessing Report ===
Original Docs: 300000
Duplicates Removed: 16358
Short Docs Removed (<50 words): 9921
Final Docs Kept: 273721
Final Dataset Size: 1.10 GB
Cleaned data saved to: /Users/zhenting/7374_LLM/clean_dataset.jsonl


## 4.2 Tokenization

* Use a transformer-compatible tokenizer (Hugging Face AutoTokenizer or similar).
* Support BPE, WordPiece, or GPT-style tokenization.
* Handle sequences longer than the model’s maximum block size via chunking.
* Maintain tokenized sequences in an eﬀicient data structure (list of lists, arrays, or tensors).

In [8]:
input_file = "/Users/zhenting/7374_LLM/Assignment_01/clean_dataset.jsonl"
output_dir = "/Users/zhenting/7374_LLM/Assignment_01/tokenized_data"
model_name = "gpt2" # Byte-Level
block_size = 1024
shard_size = 50000

os.makedirs(output_dir, exist_ok=True)

# 1. Tokenizer (Transformer-compatible)
print(f"Loading tokenizer: {model_name}...")
tokenizer = AutoTokenizer.from_pretrained(model_name)
# GPT-2 has no padding token, since sentence need to be the same length, so we use eos_token for padding
tokenizer.pad_token = tokenizer.eos_token 

def save_shard(tokens_list, shard_idx):
    """
    4.2 Requirement: Maintain tokenized sequences in an efficient data structure (tensors).
    """
    if not tokens_list:
        return []
    
    # list -> tensor for GPU
    tensor_data = torch.tensor(tokens_list, dtype=torch.long)
    
    # 4.2 Requirement: Handle sequences longer than block size via chunking.
    num_blocks = len(tensor_data) // block_size # 1024
    
    if num_blocks > 0:
        cutoff = num_blocks * block_size
        tensor_data = tensor_data[:cutoff] # Slicing
        
        # Reshape Ex: [2048] -> [2, 1024], two data, each 1024
        tensor_data = tensor_data.view(-1, block_size)
        
        save_path = os.path.join(output_dir, f"shard_{shard_idx}.pt")
        torch.save(tensor_data, save_path)
        print(f"Saved shard {shard_idx} to {save_path}: shape {tensor_data.shape}") # (50000 row, 1024 col token)
        
        remaining = tokens_list[cutoff:]
        return remaining
    
    return tokens_list


print(f"Tokenizing with {model_name} (Block size: {block_size})...")
all_tokens = []
shard_count = 0
doc_count = 0
eos_id = tokenizer.eos_token_id

total_lines = sum(1 for _ in open(input_file, encoding="utf-8"))

with open(input_file, "r", encoding="utf-8") as f:
    for line in tqdm(f, total=total_lines, desc="Tokenizing"):
        try:
            doc = json.loads(line)
            text = doc["text"]
            
            # Tokenization
            tokens = tokenizer.encode(text) + [eos_id]
            all_tokens.extend(tokens)
            doc_count += 1
            
            # Memory Management
            if len(all_tokens) > shard_size * block_size:
                all_tokens = save_shard(all_tokens, shard_count)
                shard_count += 1
                
        except json.JSONDecodeError:
            continue

if all_tokens:
    save_shard(all_tokens, shard_count)

print("\n=== Preprocessing Complete ===")
print(f"Check output directory: {output_dir}/")

Loading tokenizer: gpt2...
Tokenizing with gpt2 (Block size: 1024)...


Tokenizing:  29%|█████████▌                       | 79668/273721 [00:44<13:21, 242.13it/s]

50004
tensor([ 8117,   338,   257,  ..., 16667, 28841,   365])
tensor([[ 8117,   338,   257,  ...,    13,  2008, 12537],
        [  299,    88, 21101,  ...,    85,  4763, 47735],
        [21421,    13,   299,  ...,   717,  5545,  3652],
        ...,
        [40138,   410, 40138,  ..., 45630,   272, 14549],
        [19322,   784, 44873,  ...,  3104,   784,   556],
        [ 2516,   286,   277,  ..., 16667, 28841,   365]])
Saved shard 0 to /Users/zhenting/7374_LLM/tokenized_data/shard_0.pt: shape torch.Size([50004, 1024])


Tokenizing:  45%|██████████████▎                 | 122192/273721 [01:34<02:39, 949.94it/s]

50000
tensor([   11, 45630,   272,  ..., 23645,  3835,  7097])
tensor([[   11, 45630,   272,  ...,  2272,   329,   257],
        [ 4996,   286, 12420,  ..., 10389,  5823,   423],
        [ 4054,  2233,   284,  ..., 20835,     1,   416],
        ...,
        [ 1989,   319,   262,  ...,   318,  1900,   355],
        [50162,   272, 12480,  ...,   262,  3814,   810],
        [  262,  4618,   550,  ..., 23645,  3835,  7097]])
Saved shard 1 to /Users/zhenting/7374_LLM/tokenized_data/shard_1.pt: shape torch.Size([50000, 1024])


...

50000
tensor([39818,  1621,  2614,  ...,   257,  2408,   290])
tensor([[39818,  1621,  2614,  ..., 41899, 28936, 20840],
        [  262,  1989,    11,  ...,  7867,   416,   607],
        [ 2802,    11,  2855,  ...,   727,   285,   446],
        ...,
        [   13,   339,   373,  ...,  4695,    11,  6134],
        [   11, 19435,    11,  ...,  4441, 15962, 30162],
        [  329,  3511,    13,  ...,   257,  2408,   290]])
Saved shard 3 to /Users/zhenting/7374_LLM/tokenized_data/shard_3.pt: shape torch.Size([50000, 1024])


Tokenizing:  97%|█████████████████████████████▉ | 264308/273721 [04:03<00:09, 1037.27it/s]

50001
tensor([6283,  640,  329,  ...,   11,  286, 1781])
tensor([[ 6283,   640,   329,  ...,   447,   247,    82],
        [ 4925,   286,  1321,  ...,   437, 21361,    31],
        [12398,    78,    13,  ...,   479,  2002,    83],
        ...,
        [14169,  1636,    13,  ...,   393, 21628,    69],
        [   11,   484,  1422,  ...,   878,   465,  6626],
        [  351,   474,  1697,  ...,    11,   286,  1781]])
Saved shard 4 to /Users/zhenting/7374_LLM/tokenized_data/shard_4.pt: shape torch.Size([50001, 1024])


Tokenizing: 100%|███████████████████████████████| 273721/273721 [04:15<00:00, 1072.96it/s]


10208
tensor([   11,   345,   743,  ...,   339, 13339,   423])
tensor([[   11,   345,   743,  ...,   373, 40316, 30982],
        [  284,   262,  2642,  ...,  4706,  3636,   470],
        [  910,   644,  3022,  ...,  1176,   379,   281],
        ...,
        [ 1280,   262,  5739,  ...,   284, 23982,   884],
        [  355, 12574,  2049,  ...,  5526,  2370,   422],
        [ 3770, 29136,  8636,  ...,   339, 13339,   423]])
Saved shard 5 to /Users/zhenting/7374_LLM/tokenized_data/shard_5.pt: shape torch.Size([10208, 1024])

=== Preprocessing Complete ===
Check output directory: /Users/zhenting/7374_LLM/tokenized_data/


## 4.3 Custom Data Loader

* Implement a custom data loader in PyTorch or TensorFlow:
    - PyTorch: torch.utils.data.Dataset and DataLoader
    - TensorFlow: tf.data.Dataset pipeline
* Support batching and shuffling.
* Handle variable-length sequences with padding or truncation if needed.
* Enable iterable streaming for large datasets to avoid memory bottlenecks.

In [10]:
class GPTDataset(IterableDataset):
    def __init__(self, data_dir, shuffle=True):
        self.data_dir = data_dir
        self.shuffle = shuffle
        self.shard_paths = [
            os.path.join(data_dir, f) 
            for f in os.listdir(data_dir) 
            if f.startswith("shard_") and f.endswith(".pt")
        ]
        self.shard_paths.sort()

    def load_shard(self, path):
        print(f"Loading shard: {os.path.basename(path)}")
        data = torch.load(path) # shape: (N, 1024)
        return data

    def __iter__(self):
        """
        Core logic of the IterableDataset:
        1. Determine the reading order of Shards (Shard-level shuffling).
        2. Load one Shard into memory at a time.
        3. Yield each row (Block) from the loaded Shard individually.
        """
        # Shard Level Shuffle: Randomize the order of shard files to be processed.
        worker_info = torch.utils.data.get_worker_info()
        shard_indices = list(range(len(self.shard_paths)))
        
        if self.shuffle:
            random.shuffle(shard_indices)
        
        for idx in shard_indices:
            shard_path = self.shard_paths[idx]
            
            # Load shard data -> RAM at this point
            shard_data = self.load_shard(shard_path)
            num_samples = shard_data.shape[0]
            
            # Sample Level Shuffle
            indices = list(range(num_samples))
            if self.shuffle:
                random.shuffle(indices)
            
            # Yield samples (one by one (row))
            for i in indices:
                yield shard_data[i] # (1024,)


BATCH_SIZE = 8 # 8 rows in each training process
data_dir = "/Users/zhenting/7374_LLM/Assignment_01/tokenized_data"
dataset = GPTDataset(data_dir, shuffle=True)

# instance DataLoader using Pytorch, packaging into batch
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE)

for i, batch in enumerate(dataloader):
    print(f"Batch {i} shape: {batch.shape}")
    # torch.Size([8, 1024])
    if i >= 2: # top 3 batch 
        break

print("Data Loader test passed!")

Loading shard: shard_1.pt
Batch 0 shape: torch.Size([8, 1024])
Batch 1 shape: torch.Size([8, 1024])
Batch 2 shape: torch.Size([8, 1024])
Data Loader test passed!


/var/folders/k_/401hsb7n19s13514r80fb9h80000gn/T/ipykernel_52484/1920814641.py:14: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load(path) # shape: (N, 1024)


### sample batches of processed/tokenized data (e.g., first 5–10 blocks)

In [9]:
base_dir = "/Users/zhenting/7374_LLM/Assignment_01"
shard_filename = "tokenized_data/shard_0.pt"
output_filename = "sample_batch.pt"
load_path = os.path.join(base_dir, shard_filename)
save_path = os.path.join(base_dir, output_filename)

if os.path.exists(load_path):
    full_data = torch.load(load_path, weights_only=True)
    sample_data = full_data[:10]
    torch.save(sample_data, save_path)
    print(f"Sample Shape: {sample_data.shape}")
else:
    print(f"Error: {load_path} not found.")

Sample Shape: torch.Size([10, 1024])


In [11]:
file_path = "/Users/zhenting/7374_LLM/Assignment_01/sample_batch.pt"
data = torch.load(file_path, weights_only=True)
print(f"Shape: {data.shape}")
print(data[:5])

Shape: torch.Size([10, 1024])
tensor([[ 8117,   338,   257,  ...,    13,  2008, 12537],
        [  299,    88, 21101,  ...,    85,  4763, 47735],
        [21421,    13,   299,  ...,   717,  5545,  3652],
        [ 3702,   500,   338,  ...,   284,   513,    11],
        [  830,  2444,   290,  ..., 13289,  7898,   262]])
